In [108]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [109]:
# Load data
df = pd.read_csv("../Stud_Hansa_Sim_Flight_data.csv")

In [110]:
X = df[['Alpha', 'q', 'delta_e']].values  # Shape: (N, 3)
y = df[['CD']].values                     # Shape: (N, 1)

In [111]:
# No unsqueeze needed for already 2D data
x = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [112]:
# RBF Layer (same as before)
class RBFLayer(nn.Module):
    def __init__(self, in_features, out_features, sigma=None):
        super().__init__()
        idx = torch.randint(0, x.size(0), (out_features,))
        self.centers = nn.Parameter(x[idx], requires_grad=False)

        if sigma is None:
            d_max = torch.max(torch.cdist(self.centers, self.centers))
            sigma = d_max / torch.sqrt(torch.tensor(out_features, dtype=torch.float))
        self.sigma = sigma

    def forward(self, input):
        x_expand = input.unsqueeze(1).expand(-1, self.centers.size(0), -1)
        c_expand = self.centers.unsqueeze(0)
        dist = torch.sum((x_expand - c_expand) ** 2, dim=-1)
        return torch.exp(-dist / (2 * self.sigma ** 2))

In [113]:
# RBF Network
class RBFNetwork(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.rbf = RBFLayer(in_features, hidden_features)
        self.linear = nn.Linear(hidden_features, out_features)

    def forward(self, x):
        return self.linear(self.rbf(x))

In [114]:
# Model setup
model = RBFNetwork(in_features=3, hidden_features=50, out_features=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [115]:
# Training loop
for epoch in range(2000):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.14f}")


Epoch 0, Loss: 0.36943468451500
Epoch 100, Loss: 0.00034347150358
Epoch 200, Loss: 0.00014216419368
Epoch 300, Loss: 0.00008713266288
Epoch 400, Loss: 0.00005523994332
Epoch 500, Loss: 0.00003598028707
Epoch 600, Loss: 0.00002473911991
Epoch 700, Loss: 0.00001816444455
Epoch 800, Loss: 0.00001415939641
Epoch 900, Loss: 0.00001155424798
Epoch 1000, Loss: 0.00000972863836
Epoch 1100, Loss: 0.00000835660830
Epoch 1200, Loss: 0.00000726580447
Epoch 1300, Loss: 0.00000636455434
Epoch 1400, Loss: 0.00000560311082
Epoch 1500, Loss: 0.00000495283894
Epoch 1600, Loss: 0.00000439516452
Epoch 1700, Loss: 0.00000391616095
Epoch 1800, Loss: 0.00000350415917
Epoch 1900, Loss: 0.00000314897898


In [116]:
from sklearn.metrics import mean_squared_error, r2_score
# Final predictions
predicted = model(x).detach().numpy()
true = y.numpy()

# Compute MSE and R²
mse = mean_squared_error(true, predicted)
r2 = r2_score(true, predicted)

# Final Loss from last epoch
final_loss = loss.item()

# Print the metrics
print(f"\nFinal Loss: {final_loss:.10f}")
print(f"Mean Squared Error (MSE): {mse:.10f}")
print(f"R² Score: {r2:.10f}")



Final Loss: 0.0000028445
Mean Squared Error (MSE): 0.0000028417
R² Score: 0.7605144978
